In [1]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import DataManager

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

In [2]:
import numpy as np

In [3]:
stack = 'MD589'

patch_size = 224
half_size = patch_size/2
stride = 56

patch_dir = '/home/yuncong/CSHL_data_patches/'

In [ ]:
dm = DataManager(stack=stack, section=154, load_mask=False)

w = dm.image_width
h = dm.image_height 

ys, xs = np.meshgrid(np.arange(half_size, h-half_size, stride), np.arange(half_size, w-half_size, stride),
                     indexing='xy')

sample_locations = np.c_[xs.flat, ys.flat]

box_x = 38
box_y = 137
box_w = 355
box_h = 221
xmin = max(half_size, box_x*32)
xmax = min(w-half_size-1, (box_x+box_w)*32)
ymin = max(half_size, box_y*32)
ymax = min(h-half_size-1, (box_y+box_h)*32)

In [14]:
sec = 164

In [21]:
for sec in range(154,169):
# for sec in [165]:
    
    dm = DataManager(stack=stack, section=sec, load_mask=False)

    mask = dm.load_thumbnail_mask()
    indices_fg = np.where(mask[sample_locations[:,1]/32, sample_locations[:,0]/32])[0]
    indices_bg = np.setdiff1d(range(sample_locations.shape[0]), indices_fg)

    indices_roi = np.where(np.all(np.c_[sample_locations[:,0] > xmin, sample_locations[:,1] > ymin, 
                                        sample_locations[:,0] < xmax, sample_locations[:,1] < ymax], axis=1))[0]

    indices_roi = np.setdiff1d(indices_roi, indices_bg)
    print len(indices_roi), 'patches in ROI'

    output_dir = os.path.join(patch_dir, '%s_byROI/%04d/roi1'%(stack, sec))
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    dm._load_image(['rgb-jpg'])
    
#     import time
#     t = time.time()

    for i in indices_roi:
        x, y = sample_locations[i]
        patch = dm.image_rgb_jpg[y-half_size:y+half_size, x-half_size:x+half_size]
        cv2.imwrite(os.path.join(output_dir, '%(stack)s_%(sec)04d_%(gridp_ind)08d.jpg' %\
                                 {'stack':stack, 'sec':sec, 'gridp_ind':i}), 
                    patch[..., ::-1])

    tarball_filename = '%(stack)s_%(sec)04d_roi1.tar' % {'stack': stack, 'sec': sec}

    os.system('tar -I pigz -C %(output_dir)s -cf /home/yuncong/CSHL_data_patches/%(tarball_filename)s .' % \
              {'output_dir': output_dir, 'tarball_filename': tarball_filename})
        
#     print time.time() - t

20019 patches in ROI
/oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD589_lossless_aligned_cropped_downscaled/MD589_0154_lossless_aligned_cropped_downscaled.jpg
20218 patches in ROI
/oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD589_lossless_aligned_cropped_downscaled/MD589_0155_lossless_aligned_cropped_downscaled.jpg
20623 patches in ROI
/oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD589_lossless_aligned_cropped_downscaled/MD589_0156_lossless_aligned_cropped_downscaled.jpg
20755 patches in ROI
/oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD589_lossless_aligned_cropped_downscaled/MD589_0157_lossless_aligned_cropped_downscaled.jpg
20898 patches in ROI
/oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD589_lossless_aligned_cropped_downscaled/MD589_0158_lossless_aligned_cropped_downscaled.jpg
20831 patches in ROI
/oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD589_lossless_aligned_cropped_downscaled/MD589_0159_lossless_aligned_cropped_d